In [ ]:
import gc
import glob
import random
from collections import defaultdict
from pathlib import Path
from tempfile import gettempdir

import matplotlib as mpl
import matplotlib.pyplot as pyplot
import numpy as np
import pandas as pd
import scipy as sp
import scipy.constants
from IPython.core.display import HTML, display
from matplotlib import cm, ticker
from numpy import ma

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
COLORS = mpl.rcParams["axes.prop_cycle"].by_key()["color"]

# Toggle on/off interactive notebooks
if False:
    %matplotlib notebook
    FIGSIZE = 6
else:
    %matplotlib inline
    FIGSIZE = 12

mpl.rcParams["figure.figsize"] = [FIGSIZE, FIGSIZE / sp.constants.golden]

In [ ]:
def random_drop(df, count=100_000, inplace=False):
    ratio = count / df.index.size
    selector = list(filter(lambda _: random.random() > ratio, df.index))
    return df.drop(selector, inplace=inplace)

In [ ]:
OUTPUT_DIR = Path(gettempdir()) / "boltzmann_solver" / "leptogenesis"
if not OUTPUT_DIR.is_dir():
    OUTPUT_DIR = Path("/media/ssh/uni-josh/tmp/josh/boltzmann_solver/leptogenesis")

In [ ]:
def filter_ptcl(ptcl, t):
    if t.startswith("eq"):
        return filter(lambda x: x.startswith("("), ptcl)
    if t == "delta":
        return filter(lambda x: x.startswith("Δ"), ptcl)
    if t.startswith("sol"):
        return filter(lambda x: not x.startswith("Δ") and not x.startswith("("), ptcl)
    else:
        raise ValueError(
            f"Type '{t}' must be one of: eq[uilibrium], delta or sol[ution]."
        )

# Decay Only

## 1 Generation

In [ ]:
data = dict()
data["n"] = random_drop(pd.read_csv(OUTPUT_DIR / "decay_only" / "1gen" / "n.csv"))
data["n"]["ΔB-L"] = (1 / 3) * data["n"][
    ["ΔQ1", "ΔQ2", "ΔQ3", "Δu1", "Δu2", "Δu3", "Δd1", "Δd2", "Δd3"]
].sum(axis=1)
data["n"]["ΔB-L"] -= data["n"][["ΔL1", "ΔL2", "ΔL3", "Δe1", "Δe2", "Δe3"]].sum(axis=1)
data["-n"] = data["n"].apply(
    lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c
)
XSCALE = "log" if data["n"]["beta"].max() / data["n"]["beta"].min() > 100 else "linear"

gc.collect()

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])

In [ ]:
fig, ax = pyplot.subplots()
ax.set_title("Integration Evolution")
ax.set_yscale("log")
data["n"].plot(x="step", y="beta", ax=ax)

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)
for ax in axs:
    ax.set_xscale(XSCALE)

## Asymmetries
axs[0].set_title("asymmetries")
axs[0].set_yscale("log")
axs[0].set_ylim(bottom=1e-20, top=10)
data["n"].plot(
    x="beta", y="ΔB-L", ax=axs[0], linewidth=3, color="black", linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y="ΔB-L",
    ax=axs[0],
    linewidth=3,
    color="black",
    linestyle="dashed",
#     legend=False,
)
data["n"].plot(
    x="beta", y=["ΔH", "ΔL1", "ΔN1"], ax=axs[0], color=COLORS, linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔN1"],
    ax=axs[0],
    color=COLORS,
    linestyle="dashed",
    legend=False,
)


## Number densities
axs[1].set_title("Number Densities")
data["n"].abs().plot(
    x="beta", y=["H", "L1", "N1"], ax=axs[1], color=COLORS,
)
data["n"].abs().plot(
    x="beta",
    y=["(H)", "(L1)", "(N1)"],
    ax=axs[1],
    color=COLORS,
    linestyle="dotted",
    legend=False,
)
axs[1].set_ylim(bottom=0)

pyplot.show()

## 3 Generations

In [ ]:
data = dict()
data["n"] = random_drop(pd.read_csv(OUTPUT_DIR / "decay_only" / "3gen" / "n.csv"))
data["n"]["ΔB-L"] =(1/3) * data["n"][
    ["ΔQ1", "ΔQ2", "ΔQ3", "Δu1", "Δu2", "Δu3", "Δd1", "Δd2", "Δd3"]
].sum(axis=1)
data["n"]["ΔB-L"] -= data["n"][["ΔL1", "ΔL2", "ΔL3", "Δe1", "Δe2", "Δe3"]].sum(axis=1)
data["-n"] = data["n"].apply(
    lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c
)
XSCALE = "log" if data["n"]["beta"].max() / data["n"]["beta"].min() > 100 else "linear"

gc.collect()

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])

In [ ]:
fig, ax = pyplot.subplots()
ax.set_title("Integration Evolution")
ax.set_yscale("log")
data["n"].plot(x="step", y="beta", ax=ax)

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)
for ax in axs:
    ax.set_xscale(XSCALE)

## Asymmetries
axs[0].set_title("asymmetries")
axs[0].set_yscale("log")
axs[0].set_ylim(bottom=1e-20, top=10)
data["n"].plot(
    x="beta", y=["ΔB-L"], ax=axs[0], linewidth=3, color="black", linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y=["ΔB-L"],
    ax=axs[0],
    linewidth=3,
    color="black",
    linestyle="dashed",
    # legend=False,
)
data["n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔL2", "ΔL3", "ΔN1", "ΔN2", "ΔN3"],
    ax=axs[0],
    color=COLORS,
    linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔL2", "ΔL3", "ΔN1", "ΔN2", "ΔN3"],
    ax=axs[0],
    color=COLORS,
    linestyle="dashed",
    # legend=False,
)


## Number densities
axs[1].set_title("Number Densities")
data["n"].abs().plot(
    x="beta", y=["H", "L1", "L2", "L3", "N1", "N2", "N3"], ax=axs[1], color=COLORS,
)
data["n"].abs().plot(
    x="beta",
    y=["(H)", "(L1)", "(L2)", "(L3)", "(N1)", "(N2)", "(N3)"],
    ax=axs[1],
    color=COLORS,
    linestyle="dotted",
    # legend=False,
)

pyplot.show()

# $\Delta L = 2$ Only

## 1 Generation

In [ ]:
data = dict()
data["n"] = random_drop(pd.read_csv(OUTPUT_DIR / "washout_only" / "1gen" / "n.csv"))
data["n"]["ΔB-L"] =(1/3) * data["n"][
    ["ΔQ1", "ΔQ2", "ΔQ3", "Δu1", "Δu2", "Δu3", "Δd1", "Δd2", "Δd3"]
].sum(axis=1)
data["n"]["ΔB-L"] -= data["n"][["ΔL1", "ΔL2", "ΔL3", "Δe1", "Δe2", "Δe3"]].sum(axis=1)
data["-n"] = data["n"].apply(
    lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c
)
XSCALE = "log" if data["n"]["beta"].max() / data["n"]["beta"].min() > 100 else "linear"

gc.collect()

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])

In [ ]:
fig, ax = pyplot.subplots()
ax.set_title("Integration Evolution")
ax.set_yscale("log")
data["n"].plot(x="step", y="beta", ax=ax)

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)
for ax in axs:
    ax.set_xscale(XSCALE)

## Asymmetries
axs[0].set_title("asymmetries")
axs[0].set_yscale("log")
axs[0].set_ylim(bottom=1e-5)
data["n"].plot(
    x="beta", y="ΔB-L", ax=axs[0], linewidth=3, color="black", linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y="ΔB-L",
    ax=axs[0],
    linewidth=3,
    color="black",
    linestyle="dashed",
    # legend=False,
)
data["n"].plot(
    x="beta", y=["ΔH", "ΔL1", "ΔN1"], ax=axs[0], color=COLORS, linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔN1"],
    ax=axs[0],
    color=COLORS,
    linestyle="dashed",
    # legend=False,
)


## Number densities
axs[1].set_title("Number Densities")
data["n"].abs().plot(
    x="beta", y=["H", "L1", "N1"], ax=axs[1], color=COLORS,
)
data["n"].abs().plot(
    x="beta",
    y=["(H)", "(L1)", "(N1)"],
    ax=axs[1],
    color=COLORS,
    linestyle="dotted",
    # legend=False,
)

pyplot.show()

## 3 Generation

In [ ]:
data = dict()
data["n"] = random_drop(pd.read_csv(OUTPUT_DIR / "washout_only" / "3gen" / "n.csv"))
data["n"]["ΔB-L"] =(1/3) * data["n"][
    ["ΔQ1", "ΔQ2", "ΔQ3", "Δu1", "Δu2", "Δu3", "Δd1", "Δd2", "Δd3"]
].sum(axis=1)
data["n"]["ΔB-L"] -= data["n"][["ΔL1", "ΔL2", "ΔL3", "Δe1", "Δe2", "Δe3"]].sum(axis=1)
data["-n"] = data["n"].apply(
    lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c
)
XSCALE = "log" if data["n"]["beta"].max() / data["n"]["beta"].min() > 100 else "linear"

gc.collect()

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])

In [ ]:
fig, ax = pyplot.subplots()
ax.set_title("Integration Evolution")
ax.set_yscale("log")
data["n"].plot(x="step", y="beta", ax=ax)

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)
for ax in axs:
    ax.set_xscale(XSCALE)

## Asymmetries
axs[0].set_title("asymmetries")
axs[0].set_yscale("log")
axs[0].set_ylim(bottom=1e-5)
data["n"].plot(
    x="beta", y="ΔB-L", ax=axs[0], linewidth=3, color="black", linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y="ΔB-L",
    ax=axs[0],
    linewidth=3,
    color="black",
    linestyle="dashed",
    # legend=False,
)
data["n"].plot(
    x="beta", y=["ΔH", "ΔL1", "ΔN1"], ax=axs[0], color=COLORS, linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔN1"],
    ax=axs[0],
    color=COLORS,
    linestyle="dashed",
    # legend=False,
)


## Number densities
axs[1].set_title("Number Densities")
data["n"].abs().plot(
    x="beta", y=["H", "L1", "N1"], ax=axs[1], color=COLORS,
)
data["n"].abs().plot(
    x="beta",
    y=["(H)", "(L1)", "(N1)"],
    ax=axs[1],
    color=COLORS,
    linestyle="dotted",
    # legend=False,
)

pyplot.show()

# Decay + $\Delta L = 2$

## 1 Generation

In [ ]:
data = dict()
data["n"] = random_drop(pd.read_csv(OUTPUT_DIR / "decay_washout" / "1gen" / "n.csv"))
data["n"]["ΔB-L"] = (1 / 3) * data["n"][
    ["ΔQ1", "ΔQ2", "ΔQ3", "Δu1", "Δu2", "Δu3", "Δd1", "Δd2", "Δd3"]
].sum(axis=1)
data["n"]["ΔB-L"] -= data["n"][["ΔL1", "ΔL2", "ΔL3", "Δe1", "Δe2", "Δe3"]].sum(axis=1)
data["-n"] = data["n"].apply(
    lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c
)
XSCALE = "log" if data["n"]["beta"].max() / data["n"]["beta"].min() > 100 else "linear"

gc.collect()

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])

In [ ]:
fig, ax = pyplot.subplots()
ax.set_title("Integration Evolution")
ax.set_yscale("log")
data["n"].plot(x="step", y="beta", ax=ax)

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)
for ax in axs:
    ax.set_xscale(XSCALE)

## Asymmetries
axs[0].set_title("asymmetries")
axs[0].set_yscale("log")
axs[0].set_ylim(bottom=1e-20, top=10)
data["n"].plot(
    x="beta", y="ΔB-L", ax=axs[0], linewidth=3, color="black", linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y="ΔB-L",
    ax=axs[0],
    linewidth=3,
    color="black",
    linestyle="dashed",
    # legend=False,
)
data["n"].plot(
    x="beta", y=["ΔH", "ΔL1", "ΔN1"], ax=axs[0], color=COLORS, linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔN1"],
    ax=axs[0],
    color=COLORS,
    linestyle="dashed",
    # legend=False,
)


## Number densities
axs[1].set_title("Number Densities")
data["n"].abs().plot(
    x="beta", y=["H", "L1", "N1",], ax=axs[1], color=COLORS,
)
data["n"].abs().plot(
    x="beta",
    y=["(H)", "(L1)", "(N1)"],
    ax=axs[1],
    color=COLORS,
    linestyle="dotted",
    # legend=False,
)

pyplot.show()

## 3 Generation

In [ ]:
data = dict()
data["n"] = random_drop(pd.read_csv(OUTPUT_DIR / "decay_washout" / "3gen" / "n.csv"))
data["n"]["ΔB-L"] = (1 / 3) * data["n"][
    ["ΔQ1", "ΔQ2", "ΔQ3", "Δu1", "Δu2", "Δu3", "Δd1", "Δd2", "Δd3"]
].sum(axis=1)
data["n"]["ΔB-L"] -= data["n"][["ΔL1", "ΔL2", "ΔL3", "Δe1", "Δe2", "Δe3"]].sum(axis=1)
data["-n"] = data["n"].apply(
    lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c
)
XSCALE = "log" if data["n"]["beta"].max() / data["n"]["beta"].min() > 100 else "linear"

gc.collect()

print("Integration steps:", len(data["n"].index))
print("Final B-L:", data["n"]["ΔB-L"].iloc[-1])

In [ ]:
fig, ax = pyplot.subplots()
ax.set_title("Integration Evolution")
ax.set_yscale("log")
data["n"].plot(x="step", y="beta", ax=ax)

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)
for ax in axs:
    ax.set_xscale(XSCALE)

## Asymmetries
axs[0].set_title("asymmetries")
axs[0].set_yscale("log")
axs[0].set_ylim(bottom=1e-20, top=10)
data["n"].plot(
    x="beta", y="ΔB-L", ax=axs[0], linewidth=3, color="black", linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y="ΔB-L",
    ax=axs[0],
    linewidth=3,
    color="black",
    linestyle="dashed",
    # legend=False,
)
data["n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔL2", "ΔL3", "ΔN1", "ΔN2", "ΔN3"],
    ax=axs[0],
    color=COLORS,
    linestyle="solid",
)
data["-n"].plot(
    x="beta",
    y=["ΔH", "ΔL1", "ΔL2", "ΔL3", "ΔN1", "ΔN2", "ΔN3"],
    ax=axs[0],
    color=COLORS,
    linestyle="dashed",
    # legend=False,
)


## Number densities
axs[1].set_title("Number Densities")
data["n"].abs().plot(
    x="beta", y=["H", "L1", "L2", "L3", "N1", "N2", "N3"], ax=axs[1], color=COLORS,
)
data["n"].abs().plot(
    x="beta",
    y=["(H)", "(L1)", "(L2)", "(L3)", "(N1)", "(N2)", "(N3)"],
    ax=axs[1],
    color=COLORS,
    linestyle="dotted",
    # legend=False,
)

pyplot.show()

# Miscellaneous

## Masses and Widths

In [ ]:
data = dict()
data["mass"] = pd.read_csv(OUTPUT_DIR / "mass.csv")
data["width"] = pd.read_csv(OUTPUT_DIR / "width.csv")

data["mass beta"] = data["mass"].copy()
data["mass beta"].iloc[:, 1:] = (
    data["mass"].iloc[:, 1:].multiply(data["mass"].iloc[:, 0], axis="index")
)

data["width / mass"] = data["width"].copy()
data["width / mass"].iloc[:, 1:] = (
    data["width"].iloc[:, 1:]
    / data["mass"].iloc[:, 1:]  # .multiply(data["mass"].iloc[:, 0], axis="index")
)

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)

data["mass beta"].plot(
    x="beta", y=["A", "W", "H", "L3", "e3", "Q3", "u3", "d3"], ax=axs[0]
)
data["mass"].plot(x="beta", y=data["mass"].columns[-3:], ax=axs[1])

for ax in axs:
    ax.set_xlabel(r"Inverse Temperature [GeV$^{-1}$]")
    ax.set_xscale("log")
    ax.set_ylabel("Mass / Temperature")
    ax.set_yscale("log")
axs[-1].set_ylabel("Mass [GeV]")

pyplot.show()

In [ ]:
fig, axs = pyplot.subplots(
    nrows=2, sharex=True, figsize=[FIGSIZE, 2 * FIGSIZE / sp.constants.golden_ratio]
)

data["width / mass"].plot(x="beta", y=["H"], ax=axs[0])
data["width / mass"].plot(x="beta", y=["N1", "N2", "N3"], ax=axs[1])

for ax in axs:
    ax.set_xlabel(r"Inverse Temperature [GeV$^{-1}$]")
    ax.set_xscale("log")
    ax.set_ylabel("Width / Mass")
    ax.set_yscale("log")
# axs[-1].set_ylabel("Width [GeV]")

pyplot.show()

## Higgs Equilibrium

In [ ]:
datas = []
files = sorted(glob.glob(str(OUTPUT_DIR / "higgs_equilibrium" / "*.csv")))

for file in files:
    data = dict()
    data["n"] = pd.read_csv(file)
    data["-n"] = data["n"].apply(
        lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c
    )
    datas.append(data)
    
XSCALE = (
    "log"
    if datas[-1]["n"]["beta"].max() / datas[0]["n"]["beta"].min() > 100
    else "linear"
)

In [ ]:
fig, ax = pyplot.subplots(
    nrows=1, sharex=True, figsize=[FIGSIZE, FIGSIZE / sp.constants.golden_ratio]
)
ax.set_xscale(XSCALE)
cmap = mpl.cm.get_cmap("viridis")

## Number densities
ax.set_title("Number Densities")
for i, data in enumerate(datas):
    color = cmap(i / len(datas))
    data["n"].plot(
        x="beta", y=["H"], ax=ax, color=color, legend=(i == 0)
    )
    data["n"].plot(
        x="beta", y=["(H)"], ax=ax, color=color, linestyle="dotted", legend=False
    )

pyplot.show()

## Lepton Equilibrium

In [ ]:
datas = []
files = sorted(glob.glob(str(OUTPUT_DIR / "lepton_equilibrium" / "*.csv")))

for file in files:
    data = dict()
    data["n"] = pd.read_csv(file)
    data["-n"] = data["n"].apply(
        lambda c: c.apply(np.negative) if c.name not in ["step", "beta"] else c
    )
    datas.append(data)
    
XSCALE = (
    "log"
    if datas[-1]["n"]["beta"].max() / datas[0]["n"]["beta"].min() > 100
    else "linear"
)

In [ ]:
fig, ax = pyplot.subplots(
    nrows=1, sharex=True, figsize=[FIGSIZE, FIGSIZE / sp.constants.golden_ratio]
)
ax.set_xscale(XSCALE)
cmap = mpl.cm.get_cmap("viridis")

## Number densities
ax.set_title("Number Densities")
# ax.set_ylim(bottom=2, top=5)
for i, data in enumerate(datas):
    color = cmap(i / len(datas))
    data["n"].plot(
        x="beta", y=["L1"], ax=ax, color=color, legend=(i == 0)
    )
    data["n"].plot(
        x="beta", y=["(L1)"], ax=ax, color=color, linestyle="dotted", legend=False
    )

pyplot.show()

## Gammas

In [ ]:
data = defaultdict(dict)

for file in glob.glob(str(OUTPUT_DIR / "gamma" / "spline" / "*" / "*.csv")):
    # print(file)
    group = file.split("/")[-2]
    name = file.split("/")[-1].split(".csv")[0]
    data[group][f"{name} [Spline]"] = pd.read_csv(file)

for file in glob.glob(str(OUTPUT_DIR / "gamma" / "raw" / "*" / "*.csv")):
    group = file.split("/")[-2]
    name = file.split("/")[-1].split(".csv")[0]
    data[group][f"{name} [Raw]"] = pd.read_csv(file)

In [ ]:
for group in data.keys():
    fig, ax = pyplot.subplots()
    ax.set_xlabel("Inverse Temperature [GeV$^{-1}$]")
#     ax.set_ylabel(r"$\frac{\gamma}{n_1 H \beta}$")
    ax.set_ylabel(r"$\gamma$")

    colors = dict()
    for i, name in enumerate(data[group].keys()):
        if len(group.split()) == 3:
            if "Raw" in name:
                continue
    
        short_name = name.split("[")[0]
        if short_name in colors:
            color = colors[short_name]
        else:
            colors[short_name] = [COLORS[2 * i], COLORS[2 * i + 1]]

        ax.plot(
            data[group][name]["beta"],
            data[group][name]["gamma [normalized]"],
#             data[group][name]["gamma"],
            label=name,
            linestyle="solid" if "Spline" in name else "dotted",
            color=colors[short_name][0],
        )
        ax.legend()

    ax.set_xscale("log")
    ax.set_yscale("log")
#     ax.set_ylim(bottom=max(1e-10, ax.get_ylim()[0]))
    ax.set_ylim(bottom=1e-10, top=1e20)

    # Draw the region where process go from being fast to slow
    ax.autoscale(enable=False)
    ax.fill_between([1e-17, 1e-2], 0.1, 10, alpha=0.1, color=(0.1, 0.1, 0.1))

In [ ]:
gc.collect()